<a href="https://colab.research.google.com/github/ABHI2410/Project-6367/blob/main/machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Setup**

In [1]:
!pip install tqdm opencv-python einops tensorflow==2.13.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 98.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 117.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 51.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    F

In [2]:
import tqdm
import random
import pathlib
import itertools
import collections

import cv2
import einops
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


## **Pre Processing Data**

In [4]:
class CustomLableEncoder():
    def __init__(self,data_to_transform):
        self.label_encoder = preprocessing.LabelEncoder()
        self.df = pd.read_csv('/content/drive/MyDrive/ASL_Videos/how2sign_realigned_train.csv', sep= "	", header= 0)

    def __call__(self):
        text = []
        data = ""
        for index, row in self.df.iterrows():
            data = row['SENTENCE'][:-1].lower().split(' ')
            text = text + [i  for i in data if i not in text]
        self.label_encoder = preprocessing.LabelEncoder()
        self.label_encoder.fit(text)

    def transform(self, data_to_transform):
        return self.label_encoder(data_to_transform)

In [5]:
class KeyPointDetector():
    def __init__(self,path, frame_number):
        self.path = '/content/drive/MyDrive/ASL_Videos/'
        self.holistic = mp.solutions.holistic
        self.output_path = path.split('.')[0]
        self.file_name = frame_number

    def extract_keypoints(self,results):
        print("stated extracting points")
        pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
        face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
        lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
        rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
        final_data = np.concatenate([pose, face, lh, rh])
        np.save(str(self.path)+'/processed_data/'+str(self.file_name), final_data)
        return final_data


    def keypoints(self,image):
        mediapipe_holistic = self.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
        image.flags.writeable = False
        outcome = mediapipe_holistic.process(image)
        image.flags.writeable = True
        mediapipe_holistic.close()
        return self.extract_keypoints(outcome)



In [6]:
class FrameGenerator():
    def __int__(self):
        self.df = pd.read_csv('/content/drive/MyDrive/ASL_Videos/how2sign_realigned_train.csv', sep= "	", header= 0)
        self.video_path = '/content/drive/MyDrive/ASL_Videos/raw_videos/'

    def get_video_and_label(self):
        encoder = CustomLableEncoder()
        labels = list()
        videos = list()
        for index, row in self.df.iterrows():
            data = row['SENTENCE'][:-1].lower().split(' ')
            video = row['SENTENCE_NAME'] + ".mp4"
            labels.append(encoder.transform(data))
            if os.path.exists(self.video_path + video):
                videos.append(self.video_path + video)
            else:
                print("Video not found")
                exit()
        return videos, labels


    def __call__(self):
        video_path, labels = self.get_video_and_label()
        data_lable_pair = list(zip(video_path, labels))

        random.shuffle(data_lable_pair)
        print("called")
        for path,lable in data_lable_pair:
            video_frames = []
            cap = cv2.VideoCapture(path)
            frame_number = 0
            while cap.isOpened():
                print("video started")
                ret, frame = cap.read()
                landmarks = KeyPointDetector()
                video_frames.append(landmarks.keypoints(frame,path,frame_number))
                frame_number += 1
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            cap.release()
            cv2.destroyAllWindows()

            yield video_frames, lable




In [7]:
batch_size = 100

output_signature = (tf.TensorSpec(shape = (1662), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))
train_ds = tf.data.Dataset.from_generator(FrameGenerator(), output_signature= output_signature)

train_ds = train_ds.batch(batch_size)


In [8]:
HEIGHT = 720
WIDTH = 1280

In [9]:
class Conv3Dim(keras.layers.Layer):
    def __init__(self, filters, kernel_size, padding):
        super().__init__()
        self.seq = keras.Sequential([
            layers.Conv3D(filters = filters,
                          kernel_size = (1,kernel_size[1],kernel_size[2]),
                          padding=padding),
            layers.Conv3D(filters = filters,
                          kernel_size = (kernel_size[0],0,0),
                          padding=padding),
            layers.LSTM(64, return_sequences=True, activation='relu'),
            layers.LSTM(128, return_sequences=True, activation='relu'),
            layers.LSTM(64, return_sequences=False, activation='relu'),
        ])

    def call(self, inputs):
        return self.seq(inputs)

In [10]:
class ResidualMain(keras.layers.Layer):

  def __init__(self, filters, kernel_size):
    super().__init__()
    self.seq = keras.Sequential([
        Conv3Dim(filters=filters,
                    kernel_size=kernel_size,
                    padding='same'),
        layers.LayerNormalization(),
        layers.ReLU(),
        Conv3Dim(filters=filters,
                    kernel_size=kernel_size,
                    padding='same'),
        layers.LayerNormalization()
    ])

  def call(self, inputs):
    return self.seq(inputs)

In [11]:
class Project(keras.layers.Layer):
  def __init__(self, units):
    super().__init__()
    self.seq = keras.Sequential([
        layers.Dense(units),
        layers.LayerNormalization()
    ])

  def call(self, input):
    return self.seq(input)

In [12]:
def add_residual_block(input, filters, kernel_size):
  out = ResidualMain(filters,
                     kernel_size)(input)

  res = input
  if out.shape[-1] != input.shape[-1]:
    res = Project(out.shape[-1])(res)

  return layers.add([res, out])

In [13]:
class ResizeVideo(keras.layers.Layer):
  def __init__(self, height, width):
    super().__init__()
    self.height = height
    self.width = width
    self.resizing_layer = layers.Resizing(self.height, self.width)

  def call(self, video):
    # b stands for batch size, t stands for time, h stands for height,
    # w stands for width, and c stands for the number of channels.
    old_shape = einops.parse_shape(video, 'b t h w c')
    images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
    images = self.resizing_layer(images)
    videos = einops.rearrange(
        images, '(b t) h w c -> b t h w c',
        t = old_shape['t'])
    return videos

In [14]:
input_shape = (None, 10, HEIGHT, WIDTH, 3)
input = layers.Input(shape=(input_shape[1:]))
x = input

x = Conv3Dim(filters=16, kernel_size=(3, 7, 7), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = ResizeVideo(HEIGHT // 2, WIDTH // 2)(x)

# Block 1
x = add_residual_block(x, 16, (3, 3, 3))
x = ResizeVideo(HEIGHT // 4, WIDTH // 4)(x)

# Block 2
x = add_residual_block(x, 32, (3, 3, 3))
x = ResizeVideo(HEIGHT // 8, WIDTH // 8)(x)

# Block 3
x = add_residual_block(x, 64, (3, 3, 3))
x = ResizeVideo(HEIGHT // 16, WIDTH // 16)(x)

# Block 4
x = add_residual_block(x, 128, (3, 3, 3))

x = layers.GlobalAveragePooling3D()(x)
x = layers.Flatten()(x)
x = layers.Dense(10)(x)

model = keras.Model(input, x)

ValueError: ignored

In [ ]:
frames, label = next(iter(train_ds))
model.build(frames)

In [ ]:
keras.utils.plot_model(model, expand_nested=True, dpi=60, show_shapes=True)

In [ ]:
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001),
              metrics = ['accuracy'])

In [ ]:
history = model.fit(x = train_ds,
                    epochs = 50,
                    validation_data = val_ds)